In [1]:
#Library import
import math

import numpy as np

#OpenCV for face detection
import cv2

#os to work with images
import os

#The glob module finds all the pathnames matching a specified pattern
import glob

#dlib for face detection
import dlib

#csv to work with csv file
import csv

import pandas as pd

from keras.preprocessing import image

from skimage import exposure

from skimage import feature

from imutils import paths

import argparse

import imutils

Using TensorFlow backend.


In [2]:
#Open and store images and images paths

#Definition of the folder direction that contains all images
imgs_dir = "C:/Users/ching/Desktop/Assignment/dataset/"
attribute_list_filename = 'attribute_list.csv'
#Array, which will contain paths of the images
imgs = []
for i in range (1, 5001):
        imgs_path = os.path.join(imgs_dir + str(i) + ".png")
        #print(imgs_path)
        imgs.extend(glob.glob(imgs_path))
#imgs[0]

In [3]:
# PATH TO ALL IMAGES

global basedir, image_paths, target_size

basedir = './dataset'
images_dir = "C:/Users/ching/Desktop/Assignment/dataset/"
labels_filename = 'attribute_list.csv'

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')


# how to find frontal human faces in an image using 68 landmarks.  These are points on the face such as the corners of the mouth, along the eyebrows, on the eyes, and so forth.

# The face detector we use is made using the classic Histogram of Oriented
# Gradients (HOG) feature combined with a linear classifier, an image pyramid,
# and sliding window detection scheme.  The pose estimator was created by
# using dlib's implementation of the paper:
# One Millisecond Face Alignment with an Ensemble of Regression Trees by
# Vahid Kazemi and Josephine Sullivan, CVPR 2014
# and was trained on the iBUG 300-W face landmark dataset (see https://ibug.doc.ic.ac.uk/resources/facial-point-annotations/):
#     C. Sagonas, E. Antonakos, G, Tzimiropoulos, S. Zafeiriou, M. Pantic.
#     300 faces In-the-wild challenge: Database and results.
#     Image and Vision Computing (IMAVIS), Special Issue on Facial Landmark Localisation "In-The-Wild". 2016.


def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)

    # loop over all facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y

    # return a tuple of (x, y, w, h)
    return (x, y, w, h)


def run_dlib_shape(image):
    # in this function we load the image, detect the landmarks of the face, and then return the image and the landmarks
    # load the input image, resize it, and convert it to grayscale
    resized_image = image.astype('uint8')

    gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    gray = gray.astype('uint8')

    # detect faces in the grayscale image
    rects = detector(gray, 1)
    num_faces = len(rects)

    if num_faces == 0:
        return None, resized_image

    face_areas = np.zeros((1, num_faces))
    face_shapes = np.zeros((136, num_faces), dtype=np.int64)

    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        temp_shape = predictor(gray, rect)
        temp_shape = shape_to_np(temp_shape)

        # convert dlib's rectangle to a OpenCV-style bounding box
        # [i.e., (x, y, w, h)],
        #   (x, y, w, h) = face_utils.rect_to_bb(rect)
        (x, y, w, h) = rect_to_bb(rect)
        face_shapes[:, i] = np.reshape(temp_shape, [136])
        face_areas[0, i] = w * h
    # find largest face and keep
    dlibout = np.reshape(np.transpose(face_shapes[:, np.argmax(face_areas)]), [68, 2])

    return dlibout, resized_image

def extract_features_labels():
    """
    This funtion extracts the landmarks features for all images in the folder 'dataset/celeba'.
    It also extract the gender label for each image.
    :return:
        landmark_features:  an array containing 68 landmark points for each image in which a face was detected
        gender_labels:      an array containing the gender label (male=0 and female=1) for each image in
                            which a face was detected
    """
    image_paths = []
    for i in range (1, 5001):
            imgs_path = os.path.join(imgs_dir + str(i) + ".png")
            #print(imgs_path)
            image_paths.extend(glob.glob(imgs_path))
    #image_paths = [os.path.join(images_dir, l) for l in os.listdir(images_dir)]
    print("IMAGE PATHS")
    print(image_paths)
    target_size = None
    labels_file = open(labels_filename, 'r')
    lines = labels_file.readlines()
    #gender_labels = {line.split(',')[0] : int(line.split(',')[6]) for line in lines[2:]}
    if os.path.isdir(images_dir):
        all_features = []
        images_to_keep = []
        images_to_remove = []
        #all_labels = []
        for img_path in image_paths:
            print(img_path)
            file_name= img_path.split('.')[1].split('/')[-1]

            # load image
            img = image.img_to_array(
                image.load_img(img_path,
                               target_size=target_size,
                               interpolation='bicubic'))
            features, _ = run_dlib_shape(img)
            if features is not None:
                all_features.append(features)
                images_to_keep.append(img_path)

                #all_labels.append(gender_labels[file_name])
            else:
                images_to_remove.append(img_path)

    landmark_features = np.array(all_features)
    #gender_labels = (np.array(all_labels) + 1)/2 # simply converts the -1 into 0, so male=0 and female=1
    return landmark_features, images_to_keep, images_to_remove#, gender_labels



In [4]:
imgs_features, imgs_to_keep, imgs_to_remove = extract_features_labels()

IMAGE PATHS
['C:/Users/ching/Desktop/Assignment/dataset/1.png', 'C:/Users/ching/Desktop/Assignment/dataset/2.png', 'C:/Users/ching/Desktop/Assignment/dataset/3.png', 'C:/Users/ching/Desktop/Assignment/dataset/4.png', 'C:/Users/ching/Desktop/Assignment/dataset/5.png', 'C:/Users/ching/Desktop/Assignment/dataset/6.png', 'C:/Users/ching/Desktop/Assignment/dataset/7.png', 'C:/Users/ching/Desktop/Assignment/dataset/8.png', 'C:/Users/ching/Desktop/Assignment/dataset/9.png', 'C:/Users/ching/Desktop/Assignment/dataset/10.png', 'C:/Users/ching/Desktop/Assignment/dataset/11.png', 'C:/Users/ching/Desktop/Assignment/dataset/12.png', 'C:/Users/ching/Desktop/Assignment/dataset/13.png', 'C:/Users/ching/Desktop/Assignment/dataset/14.png', 'C:/Users/ching/Desktop/Assignment/dataset/15.png', 'C:/Users/ching/Desktop/Assignment/dataset/16.png', 'C:/Users/ching/Desktop/Assignment/dataset/17.png', 'C:/Users/ching/Desktop/Assignment/dataset/18.png', 'C:/Users/ching/Desktop/Assignment/dataset/19.png', 'C:/User

C:/Users/ching/Desktop/Assignment/dataset/2.png
C:/Users/ching/Desktop/Assignment/dataset/3.png
C:/Users/ching/Desktop/Assignment/dataset/4.png
C:/Users/ching/Desktop/Assignment/dataset/5.png
C:/Users/ching/Desktop/Assignment/dataset/6.png
C:/Users/ching/Desktop/Assignment/dataset/7.png
C:/Users/ching/Desktop/Assignment/dataset/8.png
C:/Users/ching/Desktop/Assignment/dataset/9.png
C:/Users/ching/Desktop/Assignment/dataset/10.png
C:/Users/ching/Desktop/Assignment/dataset/11.png
C:/Users/ching/Desktop/Assignment/dataset/12.png
C:/Users/ching/Desktop/Assignment/dataset/13.png
C:/Users/ching/Desktop/Assignment/dataset/14.png
C:/Users/ching/Desktop/Assignment/dataset/15.png
C:/Users/ching/Desktop/Assignment/dataset/16.png
C:/Users/ching/Desktop/Assignment/dataset/17.png
C:/Users/ching/Desktop/Assignment/dataset/18.png
C:/Users/ching/Desktop/Assignment/dataset/19.png
C:/Users/ching/Desktop/Assignment/dataset/20.png
C:/Users/ching/Desktop/Assignment/dataset/21.png
C:/Users/ching/Desktop/Assig

C:/Users/ching/Desktop/Assignment/dataset/172.png
C:/Users/ching/Desktop/Assignment/dataset/173.png
C:/Users/ching/Desktop/Assignment/dataset/174.png
C:/Users/ching/Desktop/Assignment/dataset/175.png
C:/Users/ching/Desktop/Assignment/dataset/176.png
C:/Users/ching/Desktop/Assignment/dataset/177.png
C:/Users/ching/Desktop/Assignment/dataset/178.png
C:/Users/ching/Desktop/Assignment/dataset/179.png
C:/Users/ching/Desktop/Assignment/dataset/180.png
C:/Users/ching/Desktop/Assignment/dataset/181.png
C:/Users/ching/Desktop/Assignment/dataset/182.png
C:/Users/ching/Desktop/Assignment/dataset/183.png
C:/Users/ching/Desktop/Assignment/dataset/184.png
C:/Users/ching/Desktop/Assignment/dataset/185.png
C:/Users/ching/Desktop/Assignment/dataset/186.png
C:/Users/ching/Desktop/Assignment/dataset/187.png
C:/Users/ching/Desktop/Assignment/dataset/188.png
C:/Users/ching/Desktop/Assignment/dataset/189.png
C:/Users/ching/Desktop/Assignment/dataset/190.png
C:/Users/ching/Desktop/Assignment/dataset/191.png


C:/Users/ching/Desktop/Assignment/dataset/336.png
C:/Users/ching/Desktop/Assignment/dataset/337.png
C:/Users/ching/Desktop/Assignment/dataset/338.png
C:/Users/ching/Desktop/Assignment/dataset/339.png
C:/Users/ching/Desktop/Assignment/dataset/340.png
C:/Users/ching/Desktop/Assignment/dataset/341.png
C:/Users/ching/Desktop/Assignment/dataset/342.png
C:/Users/ching/Desktop/Assignment/dataset/343.png
C:/Users/ching/Desktop/Assignment/dataset/344.png
C:/Users/ching/Desktop/Assignment/dataset/345.png
C:/Users/ching/Desktop/Assignment/dataset/346.png
C:/Users/ching/Desktop/Assignment/dataset/347.png
C:/Users/ching/Desktop/Assignment/dataset/348.png
C:/Users/ching/Desktop/Assignment/dataset/349.png
C:/Users/ching/Desktop/Assignment/dataset/350.png
C:/Users/ching/Desktop/Assignment/dataset/351.png
C:/Users/ching/Desktop/Assignment/dataset/352.png
C:/Users/ching/Desktop/Assignment/dataset/353.png
C:/Users/ching/Desktop/Assignment/dataset/354.png
C:/Users/ching/Desktop/Assignment/dataset/355.png


C:/Users/ching/Desktop/Assignment/dataset/501.png
C:/Users/ching/Desktop/Assignment/dataset/502.png
C:/Users/ching/Desktop/Assignment/dataset/503.png
C:/Users/ching/Desktop/Assignment/dataset/504.png
C:/Users/ching/Desktop/Assignment/dataset/505.png
C:/Users/ching/Desktop/Assignment/dataset/506.png
C:/Users/ching/Desktop/Assignment/dataset/507.png
C:/Users/ching/Desktop/Assignment/dataset/508.png
C:/Users/ching/Desktop/Assignment/dataset/509.png
C:/Users/ching/Desktop/Assignment/dataset/510.png
C:/Users/ching/Desktop/Assignment/dataset/511.png
C:/Users/ching/Desktop/Assignment/dataset/512.png
C:/Users/ching/Desktop/Assignment/dataset/513.png
C:/Users/ching/Desktop/Assignment/dataset/514.png
C:/Users/ching/Desktop/Assignment/dataset/515.png
C:/Users/ching/Desktop/Assignment/dataset/516.png
C:/Users/ching/Desktop/Assignment/dataset/517.png
C:/Users/ching/Desktop/Assignment/dataset/518.png
C:/Users/ching/Desktop/Assignment/dataset/519.png
C:/Users/ching/Desktop/Assignment/dataset/520.png


...

C:/Users/ching/Desktop/Assignment/dataset/4261.png
C:/Users/ching/Desktop/Assignment/dataset/4262.png
C:/Users/ching/Desktop/Assignment/dataset/4263.png
C:/Users/ching/Desktop/Assignment/dataset/4264.png
C:/Users/ching/Desktop/Assignment/dataset/4265.png
C:/Users/ching/Desktop/Assignment/dataset/4266.png
C:/Users/ching/Desktop/Assignment/dataset/4267.png
C:/Users/ching/Desktop/Assignment/dataset/4268.png
C:/Users/ching/Desktop/Assignment/dataset/4269.png
C:/Users/ching/Desktop/Assignment/dataset/4270.png
C:/Users/ching/Desktop/Assignment/dataset/4271.png
C:/Users/ching/Desktop/Assignment/dataset/4272.png
C:/Users/ching/Desktop/Assignment/dataset/4273.png
C:/Users/ching/Desktop/Assignment/dataset/4274.png
C:/Users/ching/Desktop/Assignment/dataset/4275.png
C:/Users/ching/Desktop/Assignment/dataset/4276.png
C:/Users/ching/Desktop/Assignment/dataset/4277.png
C:/Users/ching/Desktop/Assignment/dataset/4278.png
C:/Users/ching/Desktop/Assignment/dataset/4279.png
C:/Users/ching/Desktop/Assignme

C:/Users/ching/Desktop/Assignment/dataset/4424.png
C:/Users/ching/Desktop/Assignment/dataset/4425.png
C:/Users/ching/Desktop/Assignment/dataset/4426.png
C:/Users/ching/Desktop/Assignment/dataset/4427.png
C:/Users/ching/Desktop/Assignment/dataset/4428.png
C:/Users/ching/Desktop/Assignment/dataset/4429.png
C:/Users/ching/Desktop/Assignment/dataset/4430.png
C:/Users/ching/Desktop/Assignment/dataset/4431.png
C:/Users/ching/Desktop/Assignment/dataset/4432.png
C:/Users/ching/Desktop/Assignment/dataset/4433.png
C:/Users/ching/Desktop/Assignment/dataset/4434.png
C:/Users/ching/Desktop/Assignment/dataset/4435.png
C:/Users/ching/Desktop/Assignment/dataset/4436.png
C:/Users/ching/Desktop/Assignment/dataset/4437.png
C:/Users/ching/Desktop/Assignment/dataset/4438.png
C:/Users/ching/Desktop/Assignment/dataset/4439.png
C:/Users/ching/Desktop/Assignment/dataset/4440.png
C:/Users/ching/Desktop/Assignment/dataset/4441.png
C:/Users/ching/Desktop/Assignment/dataset/4442.png
C:/Users/ching/Desktop/Assignme

C:/Users/ching/Desktop/Assignment/dataset/4588.png
C:/Users/ching/Desktop/Assignment/dataset/4589.png
C:/Users/ching/Desktop/Assignment/dataset/4590.png
C:/Users/ching/Desktop/Assignment/dataset/4591.png
C:/Users/ching/Desktop/Assignment/dataset/4592.png
C:/Users/ching/Desktop/Assignment/dataset/4593.png
C:/Users/ching/Desktop/Assignment/dataset/4594.png
C:/Users/ching/Desktop/Assignment/dataset/4595.png
C:/Users/ching/Desktop/Assignment/dataset/4596.png
C:/Users/ching/Desktop/Assignment/dataset/4597.png
C:/Users/ching/Desktop/Assignment/dataset/4598.png
C:/Users/ching/Desktop/Assignment/dataset/4599.png
C:/Users/ching/Desktop/Assignment/dataset/4600.png
C:/Users/ching/Desktop/Assignment/dataset/4601.png
C:/Users/ching/Desktop/Assignment/dataset/4602.png
C:/Users/ching/Desktop/Assignment/dataset/4603.png
C:/Users/ching/Desktop/Assignment/dataset/4604.png
C:/Users/ching/Desktop/Assignment/dataset/4605.png
C:/Users/ching/Desktop/Assignment/dataset/4606.png
C:/Users/ching/Desktop/Assignme

C:/Users/ching/Desktop/Assignment/dataset/4753.png
C:/Users/ching/Desktop/Assignment/dataset/4754.png
C:/Users/ching/Desktop/Assignment/dataset/4755.png
C:/Users/ching/Desktop/Assignment/dataset/4756.png
C:/Users/ching/Desktop/Assignment/dataset/4757.png
C:/Users/ching/Desktop/Assignment/dataset/4758.png
C:/Users/ching/Desktop/Assignment/dataset/4759.png
C:/Users/ching/Desktop/Assignment/dataset/4760.png
C:/Users/ching/Desktop/Assignment/dataset/4761.png
C:/Users/ching/Desktop/Assignment/dataset/4762.png
C:/Users/ching/Desktop/Assignment/dataset/4763.png
C:/Users/ching/Desktop/Assignment/dataset/4764.png
C:/Users/ching/Desktop/Assignment/dataset/4765.png
C:/Users/ching/Desktop/Assignment/dataset/4766.png
C:/Users/ching/Desktop/Assignment/dataset/4767.png
C:/Users/ching/Desktop/Assignment/dataset/4768.png
C:/Users/ching/Desktop/Assignment/dataset/4769.png
C:/Users/ching/Desktop/Assignment/dataset/4770.png
C:/Users/ching/Desktop/Assignment/dataset/4771.png
C:/Users/ching/Desktop/Assignme

C:/Users/ching/Desktop/Assignment/dataset/4915.png
C:/Users/ching/Desktop/Assignment/dataset/4916.png
C:/Users/ching/Desktop/Assignment/dataset/4917.png
C:/Users/ching/Desktop/Assignment/dataset/4918.png
C:/Users/ching/Desktop/Assignment/dataset/4919.png
C:/Users/ching/Desktop/Assignment/dataset/4920.png
C:/Users/ching/Desktop/Assignment/dataset/4921.png
C:/Users/ching/Desktop/Assignment/dataset/4922.png
C:/Users/ching/Desktop/Assignment/dataset/4923.png
C:/Users/ching/Desktop/Assignment/dataset/4924.png
C:/Users/ching/Desktop/Assignment/dataset/4925.png
C:/Users/ching/Desktop/Assignment/dataset/4926.png
C:/Users/ching/Desktop/Assignment/dataset/4927.png
C:/Users/ching/Desktop/Assignment/dataset/4928.png
C:/Users/ching/Desktop/Assignment/dataset/4929.png
C:/Users/ching/Desktop/Assignment/dataset/4930.png
C:/Users/ching/Desktop/Assignment/dataset/4931.png
C:/Users/ching/Desktop/Assignment/dataset/4932.png
C:/Users/ching/Desktop/Assignment/dataset/4933.png
C:/Users/ching/Desktop/Assignme

In [5]:
print(np.shape(imgs_features))
print(len(imgs_to_keep))
print(np.shape(imgs_to_remove))

(4349, 68, 2)
4349
(651,)


In [6]:
#Load the attributes csv file

attribute_list = pd.read_csv(attribute_list_filename)
#attribute_list[:5]

In [7]:
#This is how to access the elements of the file
attribute_list.loc[2][5]

'1'

In [8]:
#Define a Class to contain all info (path and attributes) of all images

class Image:
    def __init__(self, image, hair_color, eyeglasses,
                smiling, young, human, landmarks, img, gray):
        self.image = image
        self.hair_color = hair_color
        self.eyeglasses = eyeglasses
        self.smiling = smiling
        self.young = young
        self.human = human
        self.landmarks = landmarks
        self.img = img
        self.gray = gray        

In [9]:
#create a list of Image objects

imageList = []
x = Image
j=0
for i in range (0,5000):
    #x.image = imgs[i]
    #x.hair_color = attribute_list.loc[i+1][1]
    #x.eyeglasses = attribute_list.loc[i+1][2]
    #x.smiling = attribute_list.loc[i+1][3]
    #x.young = attribute_list.loc[i+1][4]
    #x.human = attribute_list.loc[i+1][5]
    #print(attribute_list.loc[i+1][1])
    landmarks=[]
    if (j < len(imgs_to_keep)):
        if (imgs[i] == imgs_to_keep[j]):
            #print(j)
            landmarks = imgs_features[j]
            j = j+1
    img = cv2.imread(imgs[i])
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    hair_color = int(attribute_list.loc[i+1][1])
    eyeglasses = int(attribute_list.loc[i+1][2])
    smiling = int(attribute_list.loc[i+1][3])
    young = int(attribute_list.loc[i+1][4])
    human = int(attribute_list.loc[i+1][5])
    
    imageList.append(Image(imgs[i], hair_color, 
                          eyeglasses,
                          smiling,
                          young,
                          human,
                          landmarks,
                          img,
                          gray))
#print(imageList[3].landmarks)
print("done")

done


In [10]:
j=0
removeIndex = []
for i in range (0,5000):
    if (j < len(imgs_to_remove)):
        if (imageList[i].image == imgs_to_remove[j]):
            removeIndex.append(i)
            j = j+1
        #elif (imageList[i].hair_color == -1):
            #removeIndex.append(i)

In [11]:
removeIndex[0]

3

In [12]:
for i in range (0 , len(removeIndex)):
    #print(imgs_to_remove[i])
    os.remove(imageList[removeIndex[i]].image)
    #removeElement = imageList.pop(removeIndex[i])
    #[x - 1 for x in removeIndex]
print("done")

done


In [13]:
removeIndex.reverse()
for i in range (0 , len(removeIndex)):
    #print(removeIndex[i])
    #os.remove(imageList[removeIndex[i]].image)
    removeElement = imageList.pop(removeIndex[i])
print("done")


done


In [14]:
class ImageToProcess:
    def __init__(self, imageInfo, hog, hogImage, faceCenter, landmarkDistance, landmarkNormDistance, landmarkAngle):
        self.imageInfo = imageInfo
        self.hog = hog
        self.hogImage = hogImage
        self.faceCenter = faceCenter
        self.landmarkDistance = landmarkDistance
        self.landmarkNormDistance = landmarkNormDistance
        self.landmarkAngle = landmarkAngle

In [38]:
imageToProcessList = []
x = Image
j=0
for i in range (0,len(imageList)):
    print(i)

    # extract Histogram of Oriented Gradients from the test image and
    # predict the make of the car
    (hog, hogImage) = feature.hog(imageList[i].gray, orientations=9, pixels_per_cell=(32, 32),
                            cells_per_block=(2, 2), transform_sqrt=True, block_norm="L1", visualise=True)
 
    # visualize the HOG image
    hogImage = exposure.rescale_intensity(hogImage, out_range=(0, 255))
    hogImage = hogImage.astype("uint8")
    faceCenter = np.mean(imageList[i].landmarks,axis=0)
    
    landmarkDistance = []
    landmarkNormDistance = []
    landmarkAngle = []
    
    
    xmax = imageList[i].landmarks[0][0]
    xmin = imageList[i].landmarks[0][0]
    ymax = imageList[i].landmarks[0][1]
    ymin = imageList[i].landmarks[0][1]
    for l in range(0, len(imageList[i].landmarks)):
        if (imageList[i].landmarks[l][0] > xmax):
            xmax = imageList[i].landmarks[l][0]
        if (imageList[i].landmarks[l][0] < xmin):
            xmin = imageList[i].landmarks[l][0]
        if (imageList[i].landmarks[l][1] > ymax):
            ymax = imageList[i].landmarks[l][1]
        if (imageList[i].landmarks[l][1] < ymin):
            ymin = imageList[i].landmarks[l][1]
    maxDist = math.sqrt(pow(xmax - xmin, 2) + pow(ymax - ymin, 2))
    
    
    for j in range (0, len(imageList[i].landmarks)):
        landmarkDistance.append((math.sqrt(pow(faceCenter[0] - imageList[i].landmarks[j][0], 2) 
                                           + pow(faceCenter[1] - imageList[i].landmarks[j][1], 2))))
        
        landmarkNormDistance.append((math.sqrt(pow(faceCenter[0] - imageList[i].landmarks[j][0], 2) 
                                           + pow(faceCenter[1] - imageList[i].landmarks[j][1], 2)))/maxDist)
        
        landmarkAngle.append(((math.atan2(faceCenter[1] - imageList[i].landmarks[j][1],
                                        faceCenter[0] - imageList[i].landmarks[j][0])*360)/(2*math.pi) + 180)/360)
    
    imageToProcessList.append(ImageToProcess(imageList[i],
                                             hog,hogImage,
                                             faceCenter,
                                             landmarkDistance,
                                             landmarkNormDistance,
                                             landmarkAngle))
print("done")

0
1
2
3
4
5
6
7
8
9
10


C:\Users\ching\Anaconda3\envs\Assignment_1\lib\site-packages\skimage\feature\_hog.py:248: skimage_deprecation: Argument `visualise` is deprecated and will be changed to `visualize` in v0.16
  'be changed to `visualize` in v0.16', skimage_deprecation)


11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
2

1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069


3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708


In [39]:
from random import shuffle
shuffle(imageToProcessList)

In [40]:
trainingPart = 0.75
trainingList = imageToProcessList[0:int(trainingPart*np.size(imageToProcessList)) - 1]
testList = imageToProcessList[int(trainingPart*np.size(imageToProcessList)) - 1 :]
print(np.size(imageToProcessList))
print(np.size(trainingList))
print(np.size(testList))
print(trainingList[0].faceCenter)
print(testList[0].faceCenter)

4349
3260
1089
[ 129.33823529  158.76470588]
[ 128.82352941  160.97058824]


In [41]:

print(np.shape(trainingList[0].hog))
#print(np.shape(check_training[0].landmarkNormDistance))

(1764,)


In [77]:
train_img = [o.imageInfo.img for o in trainingList]
print(np.shape(train_img))

train_gray = [o.imageInfo.gray for o in trainingList]
print(np.shape(train_gray))

train_landmarks = [o.imageInfo.landmarks for o in trainingList]
print(np.shape(train_landmarks))

train_hog = [o.hog for o in trainingList]
print(np.shape(train_hog))

train_NormDist = [o.landmarkNormDistance for o in trainingList]
print(np.shape(train_NormDist))

train_Angle = [o.landmarkAngle for o in trainingList]
print(np.shape(train_Angle))

train_smiling = [o.imageInfo.smiling for o in trainingList]
print(np.shape(train_smiling))

train_eyeglasses = [o.imageInfo.eyeglasses for o in trainingList]
print(np.shape(train_eyeglasses))

train_hair_color = [o.imageInfo.hair_color for o in trainingList]
print(np.shape(train_hair_color))

train_young = [o.imageInfo.young for o in trainingList]
print(np.shape(train_young))

train_human = [o.imageInfo.human for o in trainingList]
print(np.shape(train_human))

print()
print()
print()

test_img = [o.imageInfo.img for o in testList]
print(np.shape(test_img))

test_gray = [o.imageInfo.gray for o in testList]
print(np.shape(test_gray))

test_landmarks = [o.imageInfo.landmarks for o in testList]
print(np.shape(test_landmarks))

test_hog = [o.hog for o in testList]
print(np.shape(test_hog))

test_NormDist = [o.landmarkNormDistance for o in testList]
print(np.shape(test_NormDist))

test_Angle = [o.landmarkAngle for o in testList]
print(np.shape(test_Angle))

test_smiling = [o.imageInfo.smiling for o in testList]
print(np.shape(test_smiling))

test_eyeglasses = [o.imageInfo.eyeglasses for o in testList]
print(np.shape(test_eyeglasses))

test_hair_color = [o.imageInfo.hair_color for o in testList]
print(np.shape(test_hair_color))

test_young = [o.imageInfo.young for o in testList]
print(np.shape(test_young))

test_human = [o.imageInfo.human for o in testList]
print(np.shape(test_human))

(3260, 256, 256, 3)
(3260, 256, 256)
(3260, 68, 2)
(3260, 1764)
(3260, 68)
(3260, 68)
(3260,)
(3260,)
(3260,)
(3260,)
(3260,)



(1089, 256, 256, 3)
(1089, 256, 256)
(1089, 68, 2)
(1089, 1764)
(1089, 68)
(1089, 68)
(1089,)
(1089,)
(1089,)
(1089,)
(1089,)


In [78]:
train_NormDist_Angle_hog = []
train_NormDist_Angle = []
for i in range (0, len(train_NormDist)):
    train_NormDist_Angle.append(np.concatenate((train_NormDist[i], train_Angle[i]), axis = 0))
    train_NormDist_Angle_hog.append(np.concatenate((train_NormDist_Angle[i], train_hog[i]), axis = 0))
print(np.shape(train_NormDist))
print(np.shape(train_Angle))
print(np.shape(train_NormDist_Angle))
print(np.shape(train_NormDist_Angle_hog))

test_NormDist_Angle_hog = []
test_NormDist_Angle = []
for i in range (0, len(test_NormDist)):
    test_NormDist_Angle.append(np.concatenate((test_NormDist[i], test_Angle[i]), axis = 0))
    test_NormDist_Angle_hog.append(np.concatenate((test_NormDist_Angle[i], test_hog[i]), axis = 0))
print(np.shape(test_NormDist))
print(np.shape(test_Angle))
print(np.shape(test_NormDist_Angle))
print(np.shape(test_NormDist_Angle_hog))


(3260, 68)
(3260, 68)
(3260, 136)
(3260, 1900)
(1089, 68)
(1089, 68)
(1089, 136)
(1089, 1900)


In [79]:
train_landmarks_flattened = []
train_landmarks_reshape = []
for i in range (0, len(train_landmarks)):
    train_landmarks_flattened.append(train_landmarks[i].flatten())
    train_landmarks_reshape.append(train_landmarks[i])
    train_landmarks_reshape[i]=np.reshape(train_landmarks_reshape[i], 136, order='F')
print(np.shape(train_landmarks))
print(np.shape(train_landmarks_flattened))
print(np.shape(train_landmarks_reshape))

test_landmarks_flattened = []
test_landmarks_reshape = []
for i in range (0, len(test_landmarks)):
    test_landmarks_flattened.append(test_landmarks[i].flatten())
    test_landmarks_reshape.append(test_landmarks[i])
    test_landmarks_reshape[i]=np.reshape(test_landmarks_reshape[i], 136, order='F')
print(np.shape(test_landmarks))
print(np.shape(test_landmarks_flattened))
print(np.shape(test_landmarks_reshape))

train_gray_flattened = []
train_gray_reshape = []
for i in range (0, len(train_gray)):
    train_gray_flattened.append(train_gray[i].flatten())
    train_gray_reshape.append(train_gray[i])
    train_gray_reshape[i]=np.reshape(train_gray_reshape[i], 65536, order='F')
print(np.shape(train_gray))
print(np.shape(train_gray_flattened))
print(np.shape(train_gray_reshape))

test_gray_flattened = []
test_gray_reshape = []
for i in range (0, len(test_gray)):
    test_gray_flattened.append(test_gray[i].flatten())
    test_gray_reshape.append(test_gray[i])
    test_gray_reshape[i]=np.reshape(test_gray_reshape[i], 65536, order='F')
print(np.shape(test_gray))
print(np.shape(test_gray_flattened))
print(np.shape(test_gray_reshape))


(3260, 68, 2)
(3260, 136)
(3260, 136)
(1089, 68, 2)
(1089, 136)
(1089, 136)
(3260, 256, 256)
(3260, 65536)
(3260, 65536)
(1089, 256, 256)
(1089, 65536)
(1089, 65536)


In [24]:
from sklearn import svm
#to choose the best hyperparameters
from sklearn.model_selection import GridSearchCV
parameters = {'kernel':('linear', 'rbf', 'poly', 'sigmoid'), 'C':[0.1, 1, 10],
             'gamma':[0.1, 0.5, 1, 10], 'degree':[1, 2, 3, 4, 5, 6],}
from sklearn.svm import SVC
svc = svm.SVC()


In [149]:
from sklearn import svm
#to choose the best hyperparameters
from sklearn.model_selection import GridSearchCV
parameters_lin = {'C':[0.1, 0.5, 1, 1.5], 'gamma':[0.1, 1, 5, 10]}
parameters_rbf = {'C':[0.1, 0.7, 1, 1.5, 3, 7, 10], 'gamma':[0.1, 1, 5, 10]}
from sklearn.svm import SVC
svc_lin = svm.SVC(kernel = 'linear')
svc_rbf = svm.SVC(kernel = 'rbf')


In [47]:
from sklearn.model_selection import cross_val_score

C = [0.1, 0.5, 1, 5, 7, 10]

mean_score = 0
best_C = C[0]
best_gamma = gamma[0]
for c in C:
    scores = cross_val_score(svm.SVC(kernel = 'linear', C = c), np.asarray(train_hog), np.asarray(train_smiling), cv=5)
    mean = np.mean(scores)
    print("Total score for C = ", c, " is ", mean)
    if (mean > mean_score):
        best_C = c
        best_gamma = g
        mean_score = mean
print(best_C)
print(mean_score)

Total score for C =  0.1  is  0.901533742331
Total score for C =  0.5  is  0.918098159509
Total score for C =  1  is  0.91963190184
Total score for C =  5  is  0.911042944785
Total score for C =  7  is  0.908588957055
Total score for C =  10  is  0.90245398773
1
0.91963190184


In [49]:
C = [1, 3, 5, 7, 10]
gamma = [0.001, 0.01, 0.1, 0.5]
mean_score = 0
best_C = C[0]
best_gamma = gamma[0]
for c in C:
    for g in gamma:
        scores = cross_val_score(svm.SVC(kernel = 'rbf', C = c, gamma = g), np.asarray(train_hog), np.asarray(train_smiling), cv=5)
        mean = np.mean(scores)
        print("Total score for C = ", c,"; gamma = ", g, " is ", mean)
        if (mean > mean_score):
            best_C = c
            best_gamma = g
            mean_score = mean
print(best_C)
print(mean_score)

Total score for C =  1 ; gamma =  0.001  is  0.783742331288
Total score for C =  1 ; gamma =  0.01  is  0.783742331288
Total score for C =  1 ; gamma =  0.1  is  0.907055214724
Total score for C =  1 ; gamma =  0.5  is  0.916257668712
Total score for C =  3 ; gamma =  0.001  is  0.783742331288
Total score for C =  3 ; gamma =  0.01  is  0.884969325153
Total score for C =  3 ; gamma =  0.1  is  0.920245398773
Total score for C =  3 ; gamma =  0.5  is  0.91717791411
Total score for C =  5 ; gamma =  0.001  is  0.783742331288
Total score for C =  5 ; gamma =  0.01  is  0.902147239264
Total score for C =  5 ; gamma =  0.1  is  0.921779141104
Total score for C =  5 ; gamma =  0.5  is  0.915030674847
Total score for C =  7 ; gamma =  0.001  is  0.783742331288
Total score for C =  7 ; gamma =  0.01  is  0.90490797546
Total score for C =  7 ; gamma =  0.1  is  0.920552147239
Total score for C =  7 ; gamma =  0.5  is  0.915337423313
Total score for C =  10 ; gamma =  0.001  is  0.783742331288
T

In [57]:
def svc_lin_params (X, y, C_val):
    C = C_val
    mean_score = 0
    best_C = C[0]
    best_gamma = gamma[0]
    for c in C:
        scores = cross_val_score(svm.SVC(kernel = 'linear', C = c, gamma = g), X, y, cv=5)
        mean = np.mean(scores)
        print("Cross-validation mean score for linear SVC with C = ", c, " is ", mean)
        if (mean > mean_score):
            best_C = c
            best_gamma = g
            mean_score = mean
    print(best_C)
    print(mean_score)
    return best_C

In [58]:
def svc_rbf_params (X, y, C_val, gamma_val):
    C = C_val
    gamma = gamma_val
    mean_score = 0
    best_C = C[0]
    best_gamma = gamma[0]
    for c in C:
        for g in gamma:
            scores = cross_val_score(svm.SVC(kernel = 'rbf', C = c, gamma = g), X, y, cv=5)
            mean = np.mean(scores)
            print("Cross-validation mean score for rbf SVC with C = ", c," and gamma = ", g, " is ", mean)
            if (mean > mean_score):
                best_C = c
                best_gamma = g
                mean_score = mean
    print(best_C)
    print(best_gamma)
    print(mean_score)
    return best_C, best_gamma

In [59]:
C_smile_lin_landmarks = svc_lin_params(np.asarray(train_landmarks_reshape),
                                       np.asarray(train_smiling),
                                       [0.001, 0.01, 0.1, 0.5])

Cross-validation mean score for linear SVC with C =  0.001  is  0.915030674847
Cross-validation mean score for linear SVC with C =  0.01  is  0.919325153374
Cross-validation mean score for linear SVC with C =  0.1  is  0.916871165644
Cross-validation mean score for linear SVC with C =  0.5  is  0.911349693252
0.01
0.919325153374


In [61]:
from sklearn.metrics import confusion_matrix
print("The C coefficient used is: ", C_smile_lin_landmarks)

svc_smile_lin_landmarks = svm.SVC(kernel = 'linear', C = C_smile_lin_landmarks)

svc_smile_lin_landmarks.fit(np.asarray(train_landmarks_reshape), np.asarray(train_smiling))

smile_pred = svc_smile_lin_landmarks.predict(np.asarray(test_landmarks_reshape))

print("Final accuracy of the linear model for smile detection, with the landmarks features is: ",
      svc_smile_lin_landmarks.score(np.asarray(test_landmarks_reshape),
                                    np.asarray(test_smiling)))
print("The confusion matrix is")
print(confusion_matrix( np.asarray(test_smiling), smile_pred))


The C coefficient used is:  0.01
Final accuracy of the linear model for smile detection, with the landmarks features is:  0.914600550964
The confusion matrix is
[[171  46]
 [ 47 825]]


In [ ]:
#C_smile_rbf_landmarks, g_smile_rbf_landmarks = svc_rbf_params(np.asarray(train_landmarks_reshape),
                                                              #np.asarray(train_smiling),
                                                              #[0.01, 0.1, 0.5, 1],
                                                              #[0.01, 0.1, 0.5, 1])

In [62]:
C_smile_lin_NormDist_Angle = svc_lin_params(np.asarray(train_NormDist_Angle),
                                            np.asarray(train_smiling),
                                            [0.1, 0.5, 1, 3, 7, 10, 15])

Cross-validation mean score for linear SVC with C =  0.1  is  0.783742331288
Cross-validation mean score for linear SVC with C =  0.5  is  0.809202453988
Cross-validation mean score for linear SVC with C =  1  is  0.862576687117
Cross-validation mean score for linear SVC with C =  3  is  0.886809815951
Cross-validation mean score for linear SVC with C =  7  is  0.897852760736
Cross-validation mean score for linear SVC with C =  10  is  0.901533742331
Cross-validation mean score for linear SVC with C =  15  is  0.907055214724
15
0.907055214724


In [65]:
print("The C coefficient used is: ", C_smile_lin_NormDist_Angle)

svc_smile_lin_NormDist_Angle = svm.SVC(kernel = 'linear', C = C_smile_lin_NormDist_Angle)

svc_smile_lin_NormDist_Angle.fit(np.asarray(train_NormDist_Angle), np.asarray(train_smiling))

smile_pred = svc_smile_lin_NormDist_Angle.predict(np.asarray(test_NormDist_Angle))

print("Final accuracy of the linear model for smile detection, with the landmark normalized disntances and angles features is: ",
      svc_smile_lin_NormDist_Angle.score(np.asarray(test_NormDist_Angle),
                                         np.asarray(test_smiling)))
print("The confusion matrix is")
print(confusion_matrix( np.asarray(test_smiling), smile_pred))

The C coefficient used is:  15
Final accuracy of the linear model for smile detection, with the landmarks features is:  0.908172635445
The confusion matrix is
[[168  49]
 [ 51 821]]


In [69]:
C_smile_rbf_NormDist_Angle, g_smile_rbf_NormDist_Angle = svc_rbf_params(np.asarray(train_NormDist_Angle),
                                                                        np.asarray(train_smiling),
                                                                        [1, 3, 5, 7, 10, 15],
                                                                        [0.1, 0.5, 1, 3, 5])

Cross-validation mean score for rbf SVC with C =  1  and gamma =  0.1  is  0.785582822086
Cross-validation mean score for rbf SVC with C =  1  and gamma =  0.5  is  0.852147239264
Cross-validation mean score for rbf SVC with C =  1  and gamma =  1  is  0.866257668712
Cross-validation mean score for rbf SVC with C =  1  and gamma =  3  is  0.890490797546
Cross-validation mean score for rbf SVC with C =  1  and gamma =  5  is  0.899079754601
Cross-validation mean score for rbf SVC with C =  3  and gamma =  0.1  is  0.818711656442
Cross-validation mean score for rbf SVC with C =  3  and gamma =  0.5  is  0.885582822086
Cross-validation mean score for rbf SVC with C =  3  and gamma =  1  is  0.898159509202
Cross-validation mean score for rbf SVC with C =  3  and gamma =  3  is  0.907668711656
Cross-validation mean score for rbf SVC with C =  3  and gamma =  5  is  0.914110429448
Cross-validation mean score for rbf SVC with C =  5  and gamma =  0.1  is  0.863190184049
Cross-validation mean 

In [95]:
print("The C coefficient used is: ", C_smile_rbf_NormDist_Angle)
print("The gamma coefficient used is: ", g_smile_rbf_NormDist_Angle)

svc_smile_rbf_NormDist_Angle = svm.SVC(kernel = 'rbf', C = C_smile_rbf_NormDist_Angle, gamma = g_smile_rbf_NormDist_Angle)

svc_smile_rbf_NormDist_Angle.fit(np.asarray(train_NormDist_Angle), np.asarray(train_smiling))

smile_pred = svc_smile_rbf_NormDist_Angle.predict(np.asarray(test_NormDist_Angle))

print("Final accuracy of the rbf model for smile detection, with the landmark normalized disntances and angles features is: ",
      svc_smile_rbf_NormDist_Angle.score(np.asarray(test_NormDist_Angle),
                                         np.asarray(test_smiling)))
print("The confusion matrix is")
print(confusion_matrix( np.asarray(test_smiling), smile_pred))

The C coefficient used is:  15
The gamma coefficient used is:  5
Final accuracy of the rbf model for smile detection, with the landmark normalized disntances and angles features is:  0.91551882461
The confusion matrix is
[[169  48]
 [ 44 828]]


In [80]:
C_smile_lin_hog = svc_lin_params(np.asarray(train_hog),
                                 np.asarray(train_smiling),
                                 [0.5, 0.7, 0.9])

Cross-validation mean score for linear SVC with C =  0.5  is  0.918098159509
Cross-validation mean score for linear SVC with C =  0.7  is  0.919325153374
Cross-validation mean score for linear SVC with C =  0.9  is  0.91963190184
0.9
0.91963190184


In [81]:
print("The C coefficient used is: ", C_smile_lin_hog)

svc_smile_lin_hog = svm.SVC(kernel = 'linear', C = C_smile_lin_hog)

svc_smile_lin_hog.fit(np.asarray(train_hog), np.asarray(train_smiling))

smile_pred = svc_smile_lin_hog.predict(np.asarray(test_hog))

print("Final accuracy of the linear model for smile detection, with the HOG features is: ",
      svc_smile_lin_hog.score(np.asarray(test_hog),
                              np.asarray(test_smiling)))
print("The confusion matrix is")
print(confusion_matrix( np.asarray(test_smiling), smile_pred))

The C coefficient used is:  0.9
Final accuracy of the linear model for smile detection, with the HOG features is:  0.926538108356
The confusion matrix is
[[179  38]
 [ 42 830]]


In [82]:
C_smile_rbf_hog, g_smile_hog = svc_rbf_params(np.asarray(train_hog),
                                              np.asarray(train_smiling),
                                              [1, 3, 5, 6],
                                              [0.05, 0.1, 0.3, 0.4])

Cross-validation mean score for rbf SVC with C =  1  and gamma =  0.05  is  0.898159509202
Cross-validation mean score for rbf SVC with C =  1  and gamma =  0.1  is  0.907055214724
Cross-validation mean score for rbf SVC with C =  1  and gamma =  0.3  is  0.915030674847
Cross-validation mean score for rbf SVC with C =  1  and gamma =  0.4  is  0.916871165644
Cross-validation mean score for rbf SVC with C =  3  and gamma =  0.05  is  0.911963190184
Cross-validation mean score for rbf SVC with C =  3  and gamma =  0.1  is  0.920245398773
Cross-validation mean score for rbf SVC with C =  3  and gamma =  0.3  is  0.921472392638
Cross-validation mean score for rbf SVC with C =  3  and gamma =  0.4  is  0.919018404908
Cross-validation mean score for rbf SVC with C =  5  and gamma =  0.05  is  0.919938650307
Cross-validation mean score for rbf SVC with C =  5  and gamma =  0.1  is  0.921779141104
Cross-validation mean score for rbf SVC with C =  5  and gamma =  0.3  is  0.918711656442
Cross-v

In [96]:
print("The C coefficient used is: ", C_smile_rbf_NormDist_Angle)
print("The gamma coefficient used is: ", g_smile_hog)

svc_smile_rbf_hog = svm.SVC(kernel = 'rbf', C = C_smile_rbf_hog, gamma = g_smile_hog)

svc_smile_rbf_hog.fit(np.asarray(train_hog), np.asarray(train_smiling))

smile_pred = svc_smile_rbf_hog.predict(np.asarray(test_hog))

print("Final accuracy of the rbf model for smile detection, with the HOG features is: ",
      svc_smile_rbf_hog.score(np.asarray(test_hog),
                              np.asarray(test_smiling)))
print("The confusion matrix is")
print(confusion_matrix( np.asarray(test_smiling), smile_pred))

The C coefficient used is:  15
The gamma coefficient used is:  0.1
Final accuracy of the rbf model for smile detection, with the HOG features is:  0.924701561065
The confusion matrix is
[[177  40]
 [ 42 830]]


In [85]:
C_smile_lin_NormDist_Angle_hog = svc_lin_params(np.asarray(train_NormDist_Angle_hog),
                                                np.asarray(train_smiling),
                                                [0.5, 0.7, 0.9])

Cross-validation mean score for linear SVC with C =  0.5  is  0.941717791411
Cross-validation mean score for linear SVC with C =  0.7  is  0.941411042945
Cross-validation mean score for linear SVC with C =  0.9  is  0.941717791411
0.5
0.941717791411


In [94]:
print("The C coefficient used is: ", C_smile_lin_NormDist_Angle_hog)

svc_smile_lin_NormDist_Angle_hog = svm.SVC(kernel = 'linear', C = C_smile_lin_NormDist_Angle_hog)

svc_smile_lin_NormDist_Angle_hog.fit(np.asarray(train_NormDist_Angle_hog), np.asarray(train_smiling))

smile_pred = svc_smile_lin_NormDist_Angle_hog.predict(np.asarray(test_NormDist_Angle_hog))

print("Final accuracy of the linear model for smile detection with the landmark normalized distances, Angles and HOG features is: ",
      svc_smile_lin_NormDist_Angle_hog.score(np.asarray(test_NormDist_Angle_hog),
                                             np.asarray(test_smiling)))
print("The confusion matrix is")
print(confusion_matrix( np.asarray(test_smiling), smile_pred))

The C coefficient used is:  0.5
Final accuracy of the linear model for smile detection, with the landmark normalized distances, Angles and HOG features is:  0.939393939394
The confusion matrix is
[[188  29]
 [ 37 835]]


In [97]:
C_smile_rbf_NormDist_Angle_hog, g_smile_NormDist_Angle_hog = svc_rbf_params(np.asarray(train_NormDist_Angle_hog),
                                                                            np.asarray(train_smiling),
                                                                            [1, 3, 5, 6],
                                                                            [0.05, 0.1, 0.3, 0.4])

Cross-validation mean score for rbf SVC with C =  1  and gamma =  0.05  is  0.923926380368
Cross-validation mean score for rbf SVC with C =  1  and gamma =  0.1  is  0.937116564417
Cross-validation mean score for rbf SVC with C =  1  and gamma =  0.3  is  0.938650306748
Cross-validation mean score for rbf SVC with C =  1  and gamma =  0.4  is  0.940490797546
Cross-validation mean score for rbf SVC with C =  3  and gamma =  0.05  is  0.939877300613
Cross-validation mean score for rbf SVC with C =  3  and gamma =  0.1  is  0.943558282209
Cross-validation mean score for rbf SVC with C =  3  and gamma =  0.3  is  0.945398773006
Cross-validation mean score for rbf SVC with C =  3  and gamma =  0.4  is  0.941411042945
Cross-validation mean score for rbf SVC with C =  5  and gamma =  0.05  is  0.943558282209
Cross-validation mean score for rbf SVC with C =  5  and gamma =  0.1  is  0.94263803681
Cross-validation mean score for rbf SVC with C =  5  and gamma =  0.3  is  0.942024539877
Cross-va

In [98]:
print("The C coefficient used is: ", C_smile_rbf_NormDist_Angle_hog)
print("The gamma coefficient used is: ", g_smile_NormDist_Angle_hog)

svc_smile_rbf_NormDist_Angle_hog = svm.SVC(kernel = 'rbf', C = C_smile_rbf_NormDist_Angle_hog,
                                           gamma = g_smile_NormDist_Angle_hog)

svc_smile_rbf_NormDist_Angle_hog.fit(np.asarray(train_NormDist_Angle_hog), np.asarray(train_smiling))

smile_pred = svc_smile_rbf_NormDist_Angle_hog.predict(np.asarray(test_NormDist_Angle_hog))

print("Final accuracy of the rbf model for smile detection with the landmark normalized distances, Angles and HOG features is: ",
      svc_smile_rbf_NormDist_Angle_hog.score(np.asarray(test_NormDist_Angle_hog),
                                             np.asarray(test_smiling)))
print("The confusion matrix is")
print(confusion_matrix( np.asarray(test_smiling), smile_pred))

The C coefficient used is:  3
The gamma coefficient used is:  0.3
Final accuracy of the rbf model for smile detection with the landmark normalized distances, Angles and HOG features is:  0.941230486685
The confusion matrix is
[[186  31]
 [ 33 839]]


In [99]:
C_glasses_lin_landmarks = svc_lin_params(np.asarray(train_landmarks_reshape),
                                         np.asarray(train_eyeglasses),
                                         [0.001, 0.01, 0.1, 0.5])

Cross-validation mean score for linear SVC with C =  0.001  is  0.863796361933
Cross-validation mean score for linear SVC with C =  0.01  is  0.884959690534
Cross-validation mean score for linear SVC with C =  0.1  is  0.877902583811
Cross-validation mean score for linear SVC with C =  0.5  is  0.876059735591
0.01
0.884959690534


In [100]:
from sklearn.metrics import confusion_matrix
print("The C coefficient used is: ", C_glasses_lin_landmarks)

svc_glasses_lin_landmarks = svm.SVC(kernel = 'linear', C = C_glasses_lin_landmarks)

svc_glasses_lin_landmarks.fit(np.asarray(train_landmarks_reshape), np.asarray(train_eyeglasses))

pred = svc_glasses_lin_landmarks.predict(np.asarray(test_landmarks_reshape))

print("Final accuracy of the linear model for eyeglasses detection, with the landmarks features is: ",
      svc_glasses_lin_landmarks.score(np.asarray(test_landmarks_reshape),
                                      np.asarray(test_eyeglasses)))
print("The confusion matrix is")
print(confusion_matrix( np.asarray(test_eyeglasses), pred))


The C coefficient used is:  0.01
Final accuracy of the linear model for eyeglasses detection, with the landmarks features is:  0.895316804408
The confusion matrix is
[[754  28]
 [ 86 221]]


In [ ]:
#C_smile_rbf_landmarks, g_smile_rbf_landmarks = svc_rbf_params(np.asarray(train_landmarks_reshape),
                                                              #np.asarray(train_smiling),
                                                              #[0.01, 0.1, 0.5, 1],
                                                              #[0.01, 0.1, 0.5, 1])

In [101]:
C_glasses_lin_NormDist_Angle = svc_lin_params(np.asarray(train_NormDist_Angle),
                                              np.asarray(train_eyeglasses),
                                              [0.1, 0.5, 1, 3, 7, 10, 15])

Cross-validation mean score for linear SVC with C =  0.1  is  0.70950959644
Cross-validation mean score for linear SVC with C =  0.5  is  0.70950959644
Cross-validation mean score for linear SVC with C =  1  is  0.710122153867
Cross-validation mean score for linear SVC with C =  3  is  0.790490877642
Cross-validation mean score for linear SVC with C =  7  is  0.818702842988
Cross-validation mean score for linear SVC with C =  10  is  0.827908604005
Cross-validation mean score for linear SVC with C =  15  is  0.839864292579
15
0.839864292579


In [102]:
print("The C coefficient used is: ", C_glasses_lin_NormDist_Angle)

svc_glasses_lin_NormDist_Angle = svm.SVC(kernel = 'linear', C = C_glasses_lin_NormDist_Angle)

svc_glasses_lin_NormDist_Angle.fit(np.asarray(train_NormDist_Angle), np.asarray(train_eyeglasses))

pred = svc_glasses_lin_NormDist_Angle.predict(np.asarray(test_NormDist_Angle))

print("Final accuracy of the linear model for eyeglasses detection, with the landmark normalized disntances and angles features is: ",
      svc_glasses_lin_NormDist_Angle.score(np.asarray(test_NormDist_Angle),
                                           np.asarray(test_eyeglasses)))
print("The confusion matrix is")
print(confusion_matrix( np.asarray(test_eyeglasses), pred))

The C coefficient used is:  15
Final accuracy of the linear model for eyeglasses detection, with the landmark normalized disntances and angles features is:  0.853994490358
The confusion matrix is
[[759  23]
 [136 171]]


In [103]:
C_glasses_rbf_NormDist_Angle, g_glasses_rbf_NormDist_Angle = svc_rbf_params(np.asarray(train_NormDist_Angle),
                                                                            np.asarray(train_eyeglasses),
                                                                            [1, 3, 5, 7, 10, 15],
                                                                            [0.1, 0.5, 1, 3, 5])

Cross-validation mean score for rbf SVC with C =  1  and gamma =  0.1  is  0.70950959644
Cross-validation mean score for rbf SVC with C =  1  and gamma =  0.5  is  0.724229789333
Cross-validation mean score for rbf SVC with C =  1  and gamma =  1  is  0.761046262639
Cross-validation mean score for rbf SVC with C =  1  and gamma =  3  is  0.802454559226
Cross-validation mean score for rbf SVC with C =  1  and gamma =  5  is  0.815024681351
Cross-validation mean score for rbf SVC with C =  3  and gamma =  0.1  is  0.70950959644
Cross-validation mean score for rbf SVC with C =  3  and gamma =  0.5  is  0.7920265062
Cross-validation mean score for rbf SVC with C =  3  and gamma =  1  is  0.814106319293
Cross-validation mean score for rbf SVC with C =  3  and gamma =  3  is  0.837424416189
Cross-validation mean score for rbf SVC with C =  3  and gamma =  5  is  0.846322490678
Cross-validation mean score for rbf SVC with C =  5  and gamma =  0.1  is  0.711043812853
Cross-validation mean scor

In [105]:
print("The C coefficient used is: ", C_glasses_rbf_NormDist_Angle)
print("The gamma coefficient used is: ", g_glasses_rbf_NormDist_Angle)

svc_glasses_rbf_NormDist_Angle = svm.SVC(kernel = 'rbf', C = C_glasses_rbf_NormDist_Angle,
                                         gamma = g_glasses_rbf_NormDist_Angle)

svc_glasses_rbf_NormDist_Angle.fit(np.asarray(train_NormDist_Angle), np.asarray(train_eyeglasses))

pred = svc_glasses_rbf_NormDist_Angle.predict(np.asarray(test_NormDist_Angle))

print("Final accuracy of the rbf model for eyeglasses detection, with the landmark normalized disntances and angles features is: ",
      svc_glasses_rbf_NormDist_Angle.score(np.asarray(test_NormDist_Angle),
                                           np.asarray(test_eyeglasses)))
print("The confusion matrix is")
print(confusion_matrix( np.asarray(test_eyeglasses), pred))

The C coefficient used is:  15
The gamma coefficient used is:  5
Final accuracy of the rbf model for eyeglasses detection, with the landmark normalized disntances and angles features is:  0.880624426079
The confusion matrix is
[[744  38]
 [ 92 215]]


In [106]:
C_glasses_lin_hog = svc_lin_params(np.asarray(train_hog),
                                   np.asarray(train_eyeglasses),
                                   [0.5, 0.7, 0.9])

Cross-validation mean score for linear SVC with C =  0.5  is  0.947544974629
Cross-validation mean score for linear SVC with C =  0.7  is  0.949690806078
Cross-validation mean score for linear SVC with C =  0.9  is  0.952144798138
0.9
0.952144798138


In [107]:
print("The C coefficient used is: ", C_glasses_lin_hog)

svc_glasses_lin_hog = svm.SVC(kernel = 'linear', C = C_smile_lin_hog)

svc_glasses_lin_hog.fit(np.asarray(train_hog), np.asarray(train_eyeglasses))

pred = svc_glasses_lin_hog.predict(np.asarray(test_hog))

print("Final accuracy of the linear model for eyeglasses detection, with the HOG features is: ",
      svc_glasses_lin_hog.score(np.asarray(test_hog),
                                np.asarray(test_eyeglasses)))
print("The confusion matrix is")
print(confusion_matrix( np.asarray(test_eyeglasses), pred))

The C coefficient used is:  0.9
Final accuracy of the linear model for eyeglasses detection, with the HOG features is:  0.949494949495
The confusion matrix is
[[779   3]
 [ 52 255]]


In [108]:
C_glasses_rbf_hog, g_glasses_hog = svc_rbf_params(np.asarray(train_hog),
                                                  np.asarray(train_eyeglasses),
                                                  [1, 3, 5, 6],
                                                  [0.05, 0.1, 0.3, 0.4])

Cross-validation mean score for rbf SVC with C =  1  and gamma =  0.05  is  0.926689800865
Cross-validation mean score for rbf SVC with C =  1  and gamma =  0.1  is  0.934666687564
Cross-validation mean score for rbf SVC with C =  1  and gamma =  0.3  is  0.938653481006
Cross-validation mean score for rbf SVC with C =  1  and gamma =  0.4  is  0.934058360797
Cross-validation mean score for rbf SVC with C =  3  and gamma =  0.05  is  0.940186286719
Cross-validation mean score for rbf SVC with C =  3  and gamma =  0.1  is  0.947852191405
Cross-validation mean score for rbf SVC with C =  3  and gamma =  0.3  is  0.948161294407
Cross-validation mean score for rbf SVC with C =  3  and gamma =  0.4  is  0.942643101623
Cross-validation mean score for rbf SVC with C =  5  and gamma =  0.05  is  0.946319388579
Cross-validation mean score for rbf SVC with C =  5  and gamma =  0.1  is  0.950305713712
Cross-validation mean score for rbf SVC with C =  5  and gamma =  0.3  is  0.949079188156
Cross-v

In [109]:
print("The C coefficient used is: ", C_glasses_rbf_NormDist_Angle)
print("The gamma coefficient used is: ", g_glasses_hog)

svc_glasses_rbf_hog = svm.SVC(kernel = 'rbf', C = C_glasses_rbf_hog, gamma = g_glasses_hog)

svc_glasses_rbf_hog.fit(np.asarray(train_hog), np.asarray(train_eyeglasses))

pred = svc_glasses_rbf_hog.predict(np.asarray(test_hog))

print("Final accuracy of the rbf model for eyeglasses detection, with the HOG features is: ",
      svc_glasses_rbf_hog.score(np.asarray(test_hog),
                                np.asarray(test_eyeglasses)))
print("The confusion matrix is")
print(confusion_matrix( np.asarray(test_eyeglasses), pred))

The C coefficient used is:  15
The gamma coefficient used is:  0.1
Final accuracy of the rbf model for eyeglasses detection, with the HOG features is:  0.951331496786
The confusion matrix is
[[777   5]
 [ 48 259]]


In [110]:
C_glasses_lin_NormDist_Angle_hog = svc_lin_params(np.asarray(train_NormDist_Angle_hog),
                                                  np.asarray(train_eyeglasses),
                                                  [0.5, 0.7, 0.9])

Cross-validation mean score for linear SVC with C =  0.5  is  0.952146210282
Cross-validation mean score for linear SVC with C =  0.7  is  0.95644399151
Cross-validation mean score for linear SVC with C =  0.9  is  0.958284013998
0.9
0.958284013998


In [111]:
print("The C coefficient used is: ", C_glasses_lin_NormDist_Angle_hog)

svc_glasses_lin_NormDist_Angle_hog = svm.SVC(kernel = 'linear', C = C_glasses_lin_NormDist_Angle_hog)

svc_glasses_lin_NormDist_Angle_hog.fit(np.asarray(train_NormDist_Angle_hog), np.asarray(train_eyeglasses))

pred = svc_glasses_lin_NormDist_Angle_hog.predict(np.asarray(test_NormDist_Angle_hog))

print("Final accuracy of the linear model for eyeglasses detection with the landmark normalized distances, Angles and HOG features is: ",
      svc_glasses_lin_NormDist_Angle_hog.score(np.asarray(test_NormDist_Angle_hog),
                                               np.asarray(test_eyeglasses)))
print("The confusion matrix is")
print(confusion_matrix( np.asarray(test_eyeglasses), pred))

The C coefficient used is:  0.9
Final accuracy of the linear model for eyeglasses detection with the landmark normalized distances, Angles and HOG features is:  0.955922865014
The confusion matrix is
[[779   3]
 [ 45 262]]


In [112]:
C_glasses_rbf_NormDist_Angle_hog, g_glasses_NormDist_Angle_hog = svc_rbf_params(np.asarray(train_NormDist_Angle_hog),
                                                                                np.asarray(train_eyeglasses),
                                                                                [1, 3, 5, 6],
                                                                                [0.05, 0.1, 0.3, 0.4])

Cross-validation mean score for rbf SVC with C =  1  and gamma =  0.05  is  0.926074893232
Cross-validation mean score for rbf SVC with C =  1  and gamma =  0.1  is  0.933439690812
Cross-validation mean score for rbf SVC with C =  1  and gamma =  0.3  is  0.936811576621
Cross-validation mean score for rbf SVC with C =  1  and gamma =  0.4  is  0.93282431054
Cross-validation mean score for rbf SVC with C =  3  and gamma =  0.05  is  0.942334943899
Cross-validation mean score for rbf SVC with C =  3  and gamma =  0.1  is  0.946934768851
Cross-validation mean score for rbf SVC with C =  3  and gamma =  0.3  is  0.944480776792
Cross-validation mean score for rbf SVC with C =  3  and gamma =  0.4  is  0.940494919968
Cross-validation mean score for rbf SVC with C =  5  and gamma =  0.05  is  0.948775733731
Cross-validation mean score for rbf SVC with C =  5  and gamma =  0.1  is  0.952453898253
Cross-validation mean score for rbf SVC with C =  5  and gamma =  0.3  is  0.948776203484
Cross-va

In [113]:
print("The C coefficient used is: ", C_glasses_rbf_NormDist_Angle_hog)
print("The gamma coefficient used is: ", g_glasses_NormDist_Angle_hog)

svc_glasses_rbf_NormDist_Angle_hog = svm.SVC(kernel = 'rbf', C = C_glasses_rbf_NormDist_Angle_hog,
                                             gamma = g_glasses_NormDist_Angle_hog)

svc_glasses_rbf_NormDist_Angle_hog.fit(np.asarray(train_NormDist_Angle_hog), np.asarray(train_eyeglasses))

pred = svc_glasses_rbf_NormDist_Angle_hog.predict(np.asarray(test_NormDist_Angle_hog))

print("Final accuracy of the rbf model for eyeglasses detection with the landmark normalized distances, Angles and HOG features is: ",
      svc_glasses_rbf_NormDist_Angle_hog.score(np.asarray(test_NormDist_Angle_hog),
                                               np.asarray(test_eyeglasses)))
print("The confusion matrix is")
print(confusion_matrix( np.asarray(test_eyeglasses), pred))

The C coefficient used is:  6
The gamma coefficient used is:  0.1
Final accuracy of the rbf model for eyeglasses detection with the landmark normalized distances, Angles and HOG features is:  0.955004591368
The confusion matrix is
[[777   5]
 [ 44 263]]
